### Recommendation for Book Rental 

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### Reading data from the CSV files

In [2]:
ratings = pd.read_csv('BX-Book-Ratings.csv',encoding='latin1')
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [3]:
books = pd.read_csv('BX-Books.csv',encoding='latin1')
books.head()

c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [4]:
users = pd.read_csv('BX-Users.csv',encoding='latin1')
users.head()

c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


#### Merging all the data files

In [6]:
data_merg.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [7]:
data = data_merg.merge(users, on = 'user_id')

In [8]:
data.head(3)

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,Location,Age
0,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"cincinnati, ohio, usa",23.0
1,2313,812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,"cincinnati, ohio, usa",23.0
2,2313,679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,"cincinnati, ohio, usa",23.0


In [9]:
drop_col = ['isbn','book_author','year_of_publication','publisher','Location','Age']
data.drop(drop_col ,axis=1,inplace=True)
data.columns

Index(['user_id', 'rating', 'book_title'], dtype='object')

In [10]:
data['user_id'] = data['user_id'].astype(str)

In [11]:
data['user_id'] = data['user_id'].str.strip()

#### Split huge size data file in to seperate chunks 

In [13]:
import numpy as np
data_split = np.array_split(data,6)

In [14]:
data = pd.DataFrame(data_split[0])
data.head()

,user_id,rating,book_title
0,2313,5,Flesh Tones: A Novel
1,2313,9,Ender's Game (Ender Wiggins Saga (Paperback))
2,2313,8,In Cold Blood (Vintage International)
3,2313,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel
4,2313,5,The Mistress of Spices


#### Group with rating counts

In [15]:
data.groupby('user_id')['rating'].count().sort_values(ascending=False).head() 

user_id
11676     11147
198711     6456
98391      5779
35859      5646
76352      3329
Name: rating, dtype: int64

In [16]:
data.groupby('user_id')['rating'].mean().sort_values(ascending=False).head()

user_id
133498    10.0
184197    10.0
157823    10.0
99085     10.0
199799    10.0
Name: rating, dtype: float64

In [17]:
user_ratings=pd.DataFrame(data.groupby('book_title')['rating'].mean())
user_ratings['num_of_ratings'] = pd.DataFrame(data.groupby('book_title')['rating'].count())
user_ratings.head()

,rating,num_of_ratings
book_title,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",0.0,1
Apple Magic (The Collector's series),0.0,1
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),0.0,1
Dark Justice,10.0,1
Final Fantasy Anthology: Official Strategy Guide (Brady Games),0.0,2


#### Creating user matrix

In [18]:
usermat = data.pivot_table(index ='book_title',columns ='user_id', values ='rating').fillna(0)
usermat.head() 

user_id,100471,100846,101226,101620,102967,10314,103190,104051,105979,106225,...,97919,98153,98168,98391,9856,98741,99085,99347,9939,99720
book_title,,,,,,,,,,,,,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apple Magic (The Collector's series),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dark Justice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Final Fantasy Anthology: Official Strategy Guide (Brady Games),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
usermat.columns

Index(['100471', '100846', '101226', '101620', '102967', '10314', '103190',
       '104051', '105979', '106225',
       ...
       '97919', '98153', '98168', '98391', '9856', '98741', '99085', '99347',
       '9939', '99720'],
      dtype='object', name='user_id', length=498)

#### User with higher rating counts

In [20]:
usermat.shape

(70986, 498)

In [21]:
user_ratings_676 = usermat['11676']
user_ratings_676.head()

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    0.0
 Apple Magic (The Collector's series)                                                                         0.0
 Clifford Visita El Hospital (Clifford El Gran Perro Colorado)                                                0.0
 Dark Justice                                                                                                 0.0
 Final Fantasy Anthology: Official Strategy Guide (Brady Games)                                               0.0
Name: 11676, dtype: float64

#### Evalauting correlation with other users

In [22]:
similar_to_676 = usermat.corrwith(user_ratings_676) 
corr_676 = pd.DataFrame(similar_to_676, columns =['Correlation']) 
corr_676.dropna(inplace = True) 
corr_676.sort_values('Correlation', ascending = False).head(5)

,Correlation
user_id,
11676,1.000000
95359,0.058432
21014,0.058184
16795,0.056526
114368,0.054439
